In [ ]:
%pip install -e git+https://github.com/saumya06/crewAI_nlsql.git@nlsql #egg=crewai
%cd '/home/wsuser/work/src/crewai'
%pip install .
%cd '/home/wsuser/work/'

In [ ]:
# %pip install crewai[tools] 
from crewai_tools import NL2SQLTool
from crewai import LLM
import os, json
from crewai import Agent, Crew, Task, Process

In [ ]:
DB_URL = "52.118.208.226"
DB_PORT = "8020"
DB_NAME = "mydb"
DB_USER = "myuser"
DB_PASSWORD = ""

WX_URL= ""
WX_API_KEY = ""
WX_PROJECT_ID = ""

In [4]:
nl2sql = NL2SQLTool(db_uri=f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_URL}:{DB_PORT}/{DB_NAME}")

In [13]:

# Initialize LLM
llm_db = LLM(
    api_key = WX_API_KEY,
    api_base = WX_URL,
    model="watsonx/meta-llama/llama-3-3-70b-instruct",
    params={
        "decoding_method": "greedy",
        "max_new_tokens": 15000,
        "temperature": 0,
        "repetition_penalty": 1.05
    }
)

# Initialize LLM
llm_ba = LLM(
    api_key = WX_API_KEY,
    api_base = WX_URL,
    model="watsonx/meta-llama/llama-3-3-70b-instruct",
    params={
        "decoding_method": "sample",
        "max_new_tokens": 15000,
        "temperature": 0.8,
        "repetition_penalty": 0.1
    }
)

def create_dbagent() -> Agent:
    return Agent(
        role = "Database Analyst",
        goal = """Buat report berdasarkan pertanyaan dari user dan informasi pada database. Selalu menuliskan nama kolom dalam format table_name."column_name" pada SQL Query""",
        backstory = """Kamu adalah seorang expert di bidang data analisis dan membuat report dari database""",
        llm = llm_db,
        allow_delegation = False,
        tools = [nl2sql]
    )

def create_researcher() -> Agent:
    return Agent(
        role = "Business Analyst",
        goal = """Analisis data disesuaikan dengan jenis pertanyaan pengguna.
        Columns:
        - TRANSACTION_TIMESTAMP: Timestamp of the transaction
        - CUSTOMER_ID: Unique identifier for the customer
        - CUSTOMER_NAME: Name of the customer
        - CUSTOMER_BIRTHDAY: Customer's birth date
        - CUSTOMER_MARRIAGE_STATUS: Marital status of the customer
        - CUSTOMER_SAVINGS_VALUE: Customer's savings amount
        - TRANSACTION_VALUE: Value of the transaction
        - TRANSACTION_TYPE: Type of transaction
        - SPENDING_LOCATION_OR_CATEGORY: Location or category of spending
        - ESTIMATED_ANNUAL_INCOME: Estimated annual income of the customer
        - TOTAL_SPENDING_LAST_MONTH: Total spending in the last month
        - MOST_COMMON_SPENDING_CATEGORY: Most common spending category for the customer
        - HAS_EXISTING_LOAN: Whether the customer has an existing loan
        - LOAN_PRODUCT_INTEREST_LEVEL: Interest level in loan products
        - CUSTOMER_SEGMENT: Customer segment
         """,
        backstory = """Kamu adalah seorang expert di bidang data analisis dan membuat report dari database""",
        llm = llm_ba,
        allow_delegation = True,
    )

def create_generation_task(agent, natural_query):
    return Task(
        description = f"""{natural_query}.""",
        expected_output = """Hasil dari database setelah running query""",
        agent = agent
    )

def create_analysis_task(agent, natural_query):
    return Task(
        description = f"""{natural_query}.""",
        expected_output = """Table atau hasil analisis dari report secara text""",
        agent=agent
    )

### Running both Analysis task and Generation task

In [14]:
query = "Berapa persentase pelanggan yang berstatus menikah, single dan bercerai di dalam dataset?"

research_agent=create_researcher()
query_agent=create_dbagent()

generation_task = create_generation_task(research_agent, query)
analysis_task = create_analysis_task(research_agent, query)
generation_crew = Crew(
    agents=[research_agent, query_agent],
    tasks=[analysis_task, generation_task],
    verbose=True
    )

In [15]:
crew_output = generation_crew.kickoff()

# Accessing the crew output
print(f"Raw Output: {crew_output.raw}")

if crew_output.json_dict:
    print(f"JSON Output: {json.dumps(crew_output.json_dict, indent=2)}")

if crew_output.pydantic:
    print(f"Pydantic Output: {crew_output.pydantic}")

# print(f"Tasks Output: {crew_output.tasks_output}")
print(f"/n/nToken Usage: {crew_output.token_usage}")

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: c489095f-b7f2-43e6-8853-27751823783d                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 5f1209c3-a916-4714-b41e-131ad372474b
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 5f1209c3-a916-4714-b41e-131ad372474b
       Status: Executing Task...
    └── 🤖 Agent: Business Analyst
            Status: In Progress

# Agent: Business Analyst
## Task: Berapa persentase pelanggan yang berstatus menikah, single dan bercerai di dalam dataset?.


🚀 Crew: crew
└── 📋 Task: 5f1209c3-a916-4714-b41e-131ad372474b
       Status: Executing Task...
    └── 🤖 Agent: Business Analyst
            Status: In Progress
        └── 🧠 Thinking...

...

🚀 Crew: crew
├── 📋 Task: 5f1209c3-a916-4714-b41e-131ad372474b
│      Assigned to: Business Analyst
│      Status: ✅ Completed
│   ├── 🤖 Agent: Business Analyst
│   │       Status: In Progress
│   │   └── 🔧 Using Ask question to coworker (5)
│   └── 🤖 Agent: Business Analyst
│           Status: ✅ Completed
│       └── 🔧 Used NL2SQLTool (42)
└── 📋 Task: 2aa17933-7e57-441d-b7a8-bff394a63100
       Status: Executing Task...
    ├── 🤖 Agent: Business Analyst
    │       Status: In Progress
    │   └── 🔧 Using Delegate work to coworker (8)
    └── 🤖 Agent: Business Analyst
            Status: ✅ Completed
        └── 🔧 Used NL2SQLTool (45)

🚀 Crew: crew
├── 📋 Task: 5f1209c3-a916-4714-b41e-131ad372474b
│      Assigned to: Business Analyst
│      Status: ✅ Completed
│   ├── 🤖 Agent: Business Analyst
│   │       Status: In Progress
│   │   └── 🔧 Using Ask question to coworker (5)
│   └── 🤖 Agent: Business Analyst
│           Status: ✅ Completed
│       └── 🔧 Used NL2SQLTool (42)
└── 📋 Task: 2aa17933-7e57-441d-b7a8-bff394a63100
       Assigned to: Business Analyst
       Status: ✅ Completed
    ├── 🤖 Agent: Business Analyst
    │       Status: In Progress
    │   └── 🔧 Using Delegate work to coworker (8)
    └── 🤖 Agent: Business Analyst
            Status: ✅ Completed
        └── 🔧 Used NL2SQLTool (45)

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 2aa17933-7e57-441d-b7a8-bff394a63100                                                                     │
│  Agent: Business Analyst                                                                                        │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: c489095f-b7f2-43e6-8853-27751823783d                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Raw Output: | Status Pernikahan | Persentase |
| --- | --- |
| Menikah | 25.25% |
| Single | 67.67% |
| Bercerai | 3.53% |
| Lain-lain | 3.55% |
/n/nToken Usage: total_tokens=12376 prompt_tokens=10910 cached_prompt_tokens=0 completion_tokens=1466 successful_requests=11
